In [4]:
import os
import pandas as pd
import numpy as np
from skimage import filters

In [5]:
move_threshold = 100 # Euclidian distance for "fast movement"
window_size = 12 # Frames for non-maximum suppression (24 per second), 
                # half of actual window size because both sides are checked

In [6]:
# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.linalg.norm(p1 - p2)

# Function to calculate Euclidean distances between consecutive rows
def calculate_distances(df):
    distances = []
    for i in range(1, len(df)):
        distance = euclidean_distance(df.iloc[i-1], df.iloc[i])
        distances.append(distance)
    return distances

def non_maximum_suppression(df, distances):
    windows = []
    indices = []
    for i in range(0, len(distances)-2):
        if distances[i] > move_threshold and distances[i+1] > move_threshold:
            max_in_window = max(distances[max(0, i-window_size):min(len(distances), i+window_size+1)])
            if distances[i] == max_in_window:
                indices.append(i)
                windows.append([max(0, i-window_size), min(len(distances), i+window_size+1)])
    return indices, windows

In [10]:
def get_movements(filepath):
    with open(filepath, 'r') as file:
        df = pd.read_csv(file)
    distances = calculate_distances(df)
    indices, windows = non_maximum_suppression(df, distances)
    derivs = pd.DataFrame(df.iloc[np.array(indices) + 1].values - df.iloc[np.array(indices) - 1].values)
    return windows, derivs

In [11]:
in_dir = "../data/2_angles"
out_dir = "../data/4_movements"

print("[")
for filename in os.listdir(in_dir):
    filepath = os.path.join(in_dir, filename)
    # with open(filepath, 'r') as file:
    #     data = json.load(file)
    # print(calculate_angles(parts_candidates))
    windows, derivs = get_movements(filepath)

# read_json(filepath)
    output_path = os.path.join(out_dir, filename)
    derivs.to_csv(output_path)
    print(f"\t{[filename, windows]},")
print("]")

[
	['Chinese12.csv', [[480, 505], [561, 586], [683, 708], [799, 824], [1388, 1413], [1413, 1438], [1506, 1531], [1589, 1614]]],
	['Chinese14.csv', [[300, 325], [320, 345], [390, 415], [442, 467], [554, 579], [639, 664], [881, 906], [995, 1020], [1067, 1089]]],
	['Chinese15.csv', [[193, 218], [326, 351], [476, 501], [540, 565], [696, 721], [759, 784], [1083, 1108], [1226, 1251], [1600, 1625]]],
	['Chinese16.csv', []],
	['Chinese17.csv', [[79, 104], [122, 147], [137, 162], [205, 230], [263, 288], [287, 312], [308, 333], [562, 587], [589, 614], [619, 644], [642, 667], [680, 705], [717, 742], [762, 787], [784, 809], [818, 843], [867, 892], [893, 918]]],
	['Chinese19.csv', [[113, 138], [187, 212], [208, 233], [229, 254], [245, 270], [283, 308], [321, 346], [336, 361], [368, 393], [407, 432], [426, 451], [439, 464], [462, 487], [477, 502], [536, 561], [552, 577], [576, 601], [618, 643], [671, 696], [696, 721], [720, 745], [743, 768], [756, 781], [769, 794], [799, 824], [844, 869], [862, 887]